In [1]:
import os
# Imports the regular expression module
import re
import datetime 

import pandas as pd
import numpy as np
import pandas_gbq
import janitor

# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

In [2]:
#################################################################################################################### Task 1

In [3]:
# column name list 
col_names =  ['file_name', 'num_rows', 'num_cards', 'num_dates']
  
# create an empty dataframe
# with columns
wedge_summary  = pd.DataFrame(columns = col_names)
  
# show the dataframe
wedge_summary

,file_name,num_rows,num_cards,num_dates


In [4]:
#creates list of files in "directory"
zip_files = os.listdir("WedgeZipOfZips/")
zip_files

['transArchive_201001_201003.zip',
 'transArchive_201004_201006.zip',
 'transArchive_201007_201009.zip',
 'transArchive_201010_201012.zip',
 'transArchive_201101_201103.zip',
 'transArchive_201104.zip',
 'transArchive_201105.zip',
 'transArchive_201106.zip',
 'transArchive_201107_201109.zip',
 'transArchive_201110_201112.zip',
 'transArchive_201201_201203.zip',
 'transArchive_201201_201203_inactive.zip',
 'transArchive_201204_201206.zip',
 'transArchive_201204_201206_inactive.zip',
 'transArchive_201207_201209.zip',
 'transArchive_201207_201209_inactive.zip',
 'transArchive_201210_201212.zip',
 'transArchive_201210_201212_inactive.zip',
 'transArchive_201301_201303.zip',
 'transArchive_201301_201303_inactive.zip',
 'transArchive_201304_201306.zip',
 'transArchive_201304_201306_inactive.zip',
 'transArchive_201307_201309.zip',
 'transArchive_201307_201309_inactive.zip',
 'transArchive_201310_201312.zip',
 'transArchive_201310_201312_inactive.zip',
 'transArchive_201401_201403.zip',
 'tr

In [5]:
#this is importing class 'Zipfile' from package 'zipfile'
from zipfile import ZipFile # usually you'd do all these imports at the beginning

In [6]:
# #extracts all zips in WedgeZipOfZips
# for zipf in zip_files :
#     with ZipFile("WedgeZipOfZips/" + zipf,'r') as zf :  
#         print(zf.namelist())
        
#         print('Extracting all the files now...')
#         zf.extractall() #this extracts directly to the directory and not a subfolder
#         print('Done!')

In [7]:
#Checking for delimiters
import csv
import io

delimiters = dict() 
#r-read,w-write,a-append
#start by reading in all the files again.
for this_zf in zip_files :
    with ZipFile("WedgeZipOfZips/" + this_zf,'r') as zf :
        zipped_files = zf.namelist()
        
        for file_name in zipped_files :
            input_file = zf.open(file_name,'r')
            input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            
            dialect = csv.Sniffer().sniff(sample=input_file.readline(),
                                      delimiters=[",",";","\t"])
            
            delimiters[file_name] = dialect.delimiter
            
            print(" ".join(["It looks like",
                           file_name,
                           "has delimiter",
                           dialect.delimiter,
                           "."]))

            input_file.close() # tidy up

It looks like transArchive_201001_201003.csv has delimiter , .
It looks like transArchive_201004_201006.csv has delimiter , .
It looks like transArchive_201007_201009.csv has delimiter , .
It looks like transArchive_201010_201012.csv has delimiter , .
It looks like transArchive_201101_201103.csv has delimiter , .
It looks like transArchive_201104.csv has delimiter , .
It looks like transArchive_201105.csv has delimiter , .
It looks like transArchive_201106.csv has delimiter , .
It looks like transArchive_201107_201109.csv has delimiter , .
It looks like transArchive_201110_201112.csv has delimiter , .
It looks like transArchive_201201_201203.csv has delimiter , .
It looks like transArchive_201201_201203_inactive.csv has delimiter ; .
It looks like transArchive_201204_201206.csv has delimiter , .
It looks like transArchive_201204_201206_inactive.csv has delimiter ; .
It looks like transArchive_201207_201209.csv has delimiter , .
It looks like transArchive_201207_201209_inactive.csv has 

In [8]:
print(delimiters)

{'transArchive_201001_201003.csv': ',', 'transArchive_201004_201006.csv': ',', 'transArchive_201007_201009.csv': ',', 'transArchive_201010_201012.csv': ',', 'transArchive_201101_201103.csv': ',', 'transArchive_201104.csv': ',', 'transArchive_201105.csv': ',', 'transArchive_201106.csv': ',', 'transArchive_201107_201109.csv': ',', 'transArchive_201110_201112.csv': ',', 'transArchive_201201_201203.csv': ',', 'transArchive_201201_201203_inactive.csv': ';', 'transArchive_201204_201206.csv': ',', 'transArchive_201204_201206_inactive.csv': ';', 'transArchive_201207_201209.csv': ',', 'transArchive_201207_201209_inactive.csv': ';', 'transArchive_201210_201212.csv': ',', 'transArchive_201210_201212_inactive.csv': ';', 'transArchive_201301_201303.csv': ',', 'transArchive_201301_201303_inactive.csv': ';', 'transArchive_201304_201306.csv': ',', 'transArchive_201304_201306_inactive.csv': ';', 'transArchive_201307_201309.csv': ',', 'transArchive_201307_201309_inactive.csv': ';', 'transArchive_201310_

In [9]:
#Checking for headers
headers = dict()

for this_zf in zip_files :
    with ZipFile("WedgeZipOfZips/" + this_zf,'r') as zf :
        zipped_files = zf.namelist()

        for file_name in zipped_files :
            input_file = zf.open(file_name,'r')
            input_file = io.TextIOWrapper(input_file,encoding="utf-8")
            
            this_delimiter = delimiters[file_name]
            
            for line in input_file :
                #print(line.strip().split(this_delimiter))
                break
            print(line)
            headers[file_name] = "datetime" in line
                        
            input_file.close() # tidy up

"datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"

"datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","ch

In [10]:
headers

{'transArchive_201001_201003.csv': True,
 'transArchive_201004_201006.csv': True,
 'transArchive_201007_201009.csv': True,
 'transArchive_201010_201012.csv': True,
 'transArchive_201101_201103.csv': True,
 'transArchive_201104.csv': True,
 'transArchive_201105.csv': True,
 'transArchive_201106.csv': True,
 'transArchive_201107_201109.csv': True,
 'transArchive_201110_201112.csv': True,
 'transArchive_201201_201203.csv': True,
 'transArchive_201201_201203_inactive.csv': True,
 'transArchive_201204_201206.csv': True,
 'transArchive_201204_201206_inactive.csv': True,
 'transArchive_201207_201209.csv': True,
 'transArchive_201207_201209_inactive.csv': True,
 'transArchive_201210_201212.csv': True,
 'transArchive_201210_201212_inactive.csv': True,
 'transArchive_201301_201303.csv': True,
 'transArchive_201301_201303_inactive.csv': True,
 'transArchive_201304_201306.csv': True,
 'transArchive_201304_201306_inactive.csv': True,
 'transArchive_201307_201309.csv': True,
 'transArchive_201307_20

In [11]:
#this creates list of file names where we moved the unzipped wedge files
unzipped_files = os.listdir("Wedge_Unzipped")
#unzipped_files = unzipped_files[1:] no longer necessary after moving no_headers folder
unzipped_files

['transArchive_201512.csv',
 'transArchive_201604.csv',
 'transArchive_201605.csv']

In [12]:
#this reads list of all csv into new variable big_wedge

files = [] #place to store list of filenames
cards = [] #list to store # unique card_nos
rows = [] #count number of rows for each file and store
dates = [] #count number of dates for each file and store

#sep=delimiters.uz_file
#go across unzipped files

for uz_file in unzipped_files :
    
    files.append(uz_file) #fills files list with file names
    print(uz_file)
    print()
    
    big_wedge = pd.read_csv('Wedge_Unzipped\\'+uz_file,sep=delimiters[uz_file]) #reads in files to big_wedge
    
    num_rows = big_wedge.count()[0] #fills rows list with count of rows
    rows.append(num_rows)
    print(num_rows)
    print()
    
    num_cards = big_wedge.card_no.nunique() #fills cards list with count of unique card nos
    cards.append(num_cards) 
    print(num_cards)
    print()
    
    big_wedge['date'] = pd.to_datetime(big_wedge['datetime']).dt.floor('D')
    
    num_dates = big_wedge.date.nunique() #fills dates list with unique dates
    dates.append(num_dates)
    print(num_dates)
    print()
    
    #break
#print(big_wedge)

transArchive_201512.csv

960017

10099

30

transArchive_201604.csv

930359

9700

30

transArchive_201605.csv



C:\Users\rsmcd\AppData\Local\Temp\ipykernel_17496\4061926461.py:17: DtypeWarning: Columns (18,36,37,41,43,44,48) have mixed types. Specify dtype option on import or set low_memory=False.
  big_wedge = pd.read_csv('Wedge_Unzipped\\'+uz_file,sep=delimiters[uz_file]) #reads in files to big_wedge


938769

9440

31



In [13]:
#assign the lists of summary data we created in the previous cell to our dataframe wedge_summary
wedge_summary.file_name = files
wedge_summary.num_cards = cards
wedge_summary.num_rows = rows
wedge_summary.num_dates = dates

In [14]:
#look at it
wedge_summary

,file_name,num_rows,num_cards,num_dates
0,transArchive_201512.csv,960017,10099,30
1,transArchive_201604.csv,930359,9700,30
2,transArchive_201605.csv,938769,9440,31


In [15]:
##################################################################################### now we need to upload the data to GBQ

In [16]:
#building the private key
service_path = "C:\\Users\\rsmcd\\OneDrive\\Desktop\\MSBA Fall 2022\\" #path to json file
service_file = 'reese-msba-9558fdd20984.json' #name of json file
gbq_proj_id = 'reese-msba' #name of project

#creats single variable that leads to json file
private_key =service_path + service_file  

In [17]:
# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [18]:
# And finally we establish our connection
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [19]:
#look at list of data sets in client
for item in client.list_datasets() : 
    print(item.full_dataset_id)

reese-msba:dram_shop
reese-msba:wedge_transactions


In [59]:
gbq_proj_id = 'reese-msba'
dataset_id = 'wedge_transactions'
unzipped_files = os.listdir("Wedge_Unzipped")
unzipped_files

for uz_file in unzipped_files :
    
    big_wedge = pd.read_csv('Wedge_Unzipped\\'+uz_file,sep=delimiters[uz_file], encoding = "utf-8") #reads in files to big_wedge using file specific delimiters
    big_wedge = big_wedge.replace(r'\N', np.nan).replace(r' ', np.nan) #replaces all \N and ' ' with gbq ready null values
    for idx, column in enumerate(big_wedge) : 
        if big_wedge[column].dtypes == object : #converts all object columns to strings and replaces 'nan' with gbq ready null values
            big_wedge = big_wedge.astype({column :'str'}).replace('nan', np.nan)
        if big_wedge[column].dtypes == "int64" : #converts all integer columns to floats
            big_wedge = big_wedge.astype({column :'float'})
    big_wedge['datetime'] = pd.to_datetime(big_wedge['datetime']) #converts datetime columns to timestamp
    cols = ['wicable','taxexempt','percentDiscount','receipt','match_id','local','organic','itemstatus','tenderstatus'] #list of columns to convert to float
    for idx, col in enumerate(cols) :
        big_wedge[col] = pd.to_numeric(big_wedge[col]) #converts cols to float
    cols2 = ['memType','staff','batchHeaderID','display'] #list of columns to convert to boolean
    for idx, col in enumerate(cols2) :
        big_wedge[col] = big_wedge[col].map({1:True, 0:False}).astype('boolean') #maps 1 and 0 to True and False, then converts to boolean while preserving null values
#     break

#uploads all to gbq
    table_name, _ = uz_file.split(".")
    table_id = ".".join([gbq_proj_id,dataset_id,table_name])
    pandas_gbq.to_gbq(big_wedge, table_id, project_id=gbq_proj_id,if_exists="replace")
#     break    

###should wicable, taxexempt, local  be boolean???

100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
C:\Users\rsmcd\AppData\Local\Temp\ipykernel_17496\3338158786.py:8: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  big_wedge = pd.read_csv('Wedge_Unzipped\\'+uz_file,sep=delimiters[uz_file], encoding = "utf-8") #reads in files to big_wedge using file specific delimiters
100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
C:\Users\rsmcd\AppData\Local\Temp\ipykernel_17496\3338158786.py:8: DtypeWarning: Columns (42) have mixed types. Specify dtype option on import or set low_memory=False.
  big_wedge = pd.read_csv('Wedge_Unzipped\\'+uz_file,sep=delimiters[uz_file], encoding = "utf-8") #reads in files to big_wedge using file specific delimiters
100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:0

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 64.01it/s]
C:\Users\rsmcd\AppData\Local\Temp\ipykernel_17496\3338158786.py:8: DtypeWarning: Columns (43) have mixed types. Specify dtype option on import or set low_memory=False.
  big_wedge = pd.read_csv('Wedge_Unzipped\\'+uz_file,sep=delimiters[uz_file], encoding = "utf-8") #reads in files to big_wedge using file specific delimiters
100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<?, ?it/s]
C:\Users\rsmcd\AppData\Local\Temp\ipykernel_17496\3338158786.py:8: DtypeWarning: Columns (33,43) have mixed types. Specify dtype option on import or set low_memory=False.
  big_wedge = pd.read_csv('Wedge_Unzipped\\'+uz_file,sep=delimiters[uz_file], encoding = "utf-8") #reads in files to big_wedge using file specific delimiters
100%|████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [0

In [ ]:
# # Code used to deal with header problem: must eventually figure out how to iterate through these with for loop
# # First, but after extraction, had to move all files with no headers to no_headers folder within Wedge_Unzipped
# # Then these lines of code dealt with each file one by one and put them in the main directory
# # Then had to move all files from main directory back into the Wedge_Unzipped folder
# # Then moved no_headers folder out of Wedge_Unzipped

# # Read csv file to a DataFrame
# transArchive_201511 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201511.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201511.to_csv(
#     "transArchive_201511.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201512 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201512.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201512.to_csv(
#     "transArchive_201512.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201601 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201601.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201601.to_csv(
#     "transArchive_201601.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201602 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201602.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201602.to_csv(
#     "transArchive_201602.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201603 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201603.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201603.to_csv(
#     "transArchive_201603.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201604 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201604.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201604.to_csv(
#     "transArchive_201604.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201605 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201605.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201605.to_csv(
#     "transArchive_201605.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201606 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201606.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201606.to_csv(
#     "transArchive_201606.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201607 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201607.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201607.to_csv(
#     "transArchive_201607.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201608 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201608.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201608.to_csv(
#     "transArchive_201608.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201609 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201609.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201609.to_csv(
#     "transArchive_201609.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201610 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201610.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201610.to_csv(
#     "transArchive_201610.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201611 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201611.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201611.to_csv(
#     "transArchive_201611.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201612 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201612.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201612.to_csv(
#     "transArchive_201612.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)

# # Read csv file to a DataFrame
# transArchive_201701 = pd.read_csv('Wedge_Unzipped\\no_headers\\transArchive_201701.csv', header = None)
# # Write DataFrame to csv file, but with header
# transArchive_201701.to_csv(
#     "transArchive_201701.csv",
#     header=["datetime","register_no","emp_no","trans_no","upc","description","trans_type","trans_subtype","trans_status","department","quantity","Scale","cost","unitPrice","total","regPrice","altPrice","tax","taxexempt","foodstamp","wicable","discount","memDiscount","discountable","discounttype","voided","percentDiscount","ItemQtty","volDiscType","volume","VolSpecial","mixMatch","matched","memType","staff","numflag","itemstatus","tenderstatus","charflag","varflag","batchHeaderID","local","organic","display","receipt","card_no","store","branch","match_id","trans_id"],
#     index=False)